# Comenzamos importando todo aquello que necesitemos

In [24]:
import pandas as pd
import numpy as np

# Transformación de variables nominales

Los atributos nominales consisten en valores categóricos discretos sin noción o sentido de orden entre ellos. La idea aquí es transformar estos atributos en un formato numérico más representativo que se pueda entender fácilmente mediante el código y que nuestro algoritmo pueda procesar. Veamos un nuevo conjunto de datos relacionados con las ventas de videojuegos.

In [25]:
vg_df = pd.read_csv('../../data/vgsales.csv', encoding='utf-8')
vg_df[['Name', 'Platform', 'Year', 'Genre', 'Publisher']].iloc[0:7]

,Name,Platform,Year,Genre,Publisher
0,Wii Sports,Wii,2006.0,Sports,Nintendo
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo
5,Tetris,GB,1989.0,Puzzle,Nintendo
6,New Super Mario Bros.,DS,2006.0,Platform,Nintendo


### EJERCICIO

Obtén la lista de videojuegos únicos. ¿Cuántos son? ¿Cuántos registros tenemos en total en nuestro dataset? 

In [26]:
print(len(vg_df['Name'].unique()))
print(vg_df.shape[0])

11493
16598


¿Te parecen muchos? ¿Pocos? ¿Lo considerarías una variable categórica? Prueba a obtener los distintos géneros, ¿podría encajar en lo que estamos buscando?

In [27]:
print(len(vg_df['Genre'].unique()))
generos = vg_df['Genre'].unique()
generos

12


array(['Sports', 'Platform', 'Racing', 'Role-Playing', 'Puzzle', 'Misc',
       'Shooter', 'Simulation', 'Action', 'Fighting', 'Adventure',
       'Strategy'], dtype=object)

Guarda la lista con los diferentes géneros en una nueva variable llamada ``generos``.

### Label Encoder

Una vez hemos identificado los valores de la variable categórica, tendremos que convertir esos datos a algo interpretable por el algoritmo, es decir, datos numéricos. Para ello, podemos utiliza el LabelEncoder, que convertirá la variable categórica en numérica, asignándole un número entero que se corresponda con un mapeo que ha creado entre la categoría y un número entero (que dependerá del orden de columna que tenga).

Enlace a la [documentación](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html?highlight=labelencoder).

La interfaz será similar a lo que hemos visto con el resto de objetos de ``scikit-learn``:

In [28]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(generos)

LabelEncoder()

In [29]:
# Podemos acceder a ciertos atributos, como las categorías que está evaluando:
le.classes_

array(['Action', 'Adventure', 'Fighting', 'Misc', 'Platform', 'Puzzle',
       'Racing', 'Role-Playing', 'Shooter', 'Simulation', 'Sports',
       'Strategy'], dtype=object)

In [30]:
# Y lo que nos devuelve si transformamos:
le.transform(le.classes_)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

In [31]:
le.transform(['Action'])

array([0], dtype=int64)

In [32]:
vg_df['Genre_LE'] = np.arange(0, 16598)

In [33]:
vg_df['Genre']

0              Sports
1            Platform
2              Racing
3              Sports
4        Role-Playing
             ...     
16593        Platform
16594         Shooter
16595          Racing
16596          Puzzle
16597        Platform
Name: Genre, Length: 16598, dtype: object

### EJERCICIO

Utiliza lo que acabas de ver para crearte una nueva columna "Genre_LE" que represente la columna "Genre" pero en formato numérico:

In [34]:
le.transform(vg_df['Genre'])
vg_df['Genre_LE'] = le.transform(vg_df['Genre'])
vg_df[['Genre', 'Genre_LE']]

,Genre,Genre_LE
0,Sports,10
1,Platform,4
2,Racing,6
3,Sports,10
4,Role-Playing,7
...,...,...
16593,Platform,4
16594,Shooter,8
16595,Racing,6
16596,Puzzle,5


### EJERCICIO

Como has visto, esta forma de etiquetar los datos establece cierto orden de relación entre los datos, lo que puede que no nos guste (la mayoría de veces será algo negativo, pues las variables categóricas no tienen relación medible directamente entre sí). Hay alguna forma de codificar estos valores sin que nos imponga este orden, ¿se te ocurre cómo hacerlo? Impleméntalo.

In [35]:
df_ohe = pd.get_dummies(vg_df[['Genre']])
df_ohe.dtypes

Genre_Action          uint8
Genre_Adventure       uint8
Genre_Fighting        uint8
Genre_Misc            uint8
Genre_Platform        uint8
Genre_Puzzle          uint8
Genre_Racing          uint8
Genre_Role-Playing    uint8
Genre_Shooter         uint8
Genre_Simulation      uint8
Genre_Sports          uint8
Genre_Strategy        uint8
dtype: object

In [36]:
# vg_df.join(df_ohe)
vg_df.merge(df_ohe, left_index=True, right_index=True)

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,...,Genre_Fighting,Genre_Misc,Genre_Platform,Genre_Puzzle,Genre_Racing,Genre_Role-Playing,Genre_Shooter,Genre_Simulation,Genre_Sports,Genre_Strategy
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,...,0,0,0,0,0,0,0,0,1,0
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,...,0,0,1,0,0,0,0,0,0,0
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,...,0,0,0,0,1,0,0,0,0,0
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,...,0,0,0,0,0,0,0,0,1,0
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16593,16596,Woody Woodpecker in Crazy Castle 5,GBA,2002.0,Platform,Kemco,0.01,0.00,0.00,0.00,...,0,0,1,0,0,0,0,0,0,0
16594,16597,Men in Black II: Alien Escape,GC,2003.0,Shooter,Infogrames,0.01,0.00,0.00,0.00,...,0,0,0,0,0,0,1,0,0,0
16595,16598,SCORE International Baja 1000: The Official Game,PS2,2008.0,Racing,Activision,0.00,0.00,0.00,0.00,...,0,0,0,0,1,0,0,0,0,0
16596,16599,Know How 2,DS,2010.0,Puzzle,7G//AMES,0.00,0.01,0.00,0.00,...,0,0,0,1,0,0,0,0,0,0


# Transformando variables Ordinales

Los atributos ordinales son atributos categóricos con un sentido de orden entre los valores. Consideremos el conjunto de datos de Pokémon. Centrémonos más específicamente en el atributo Tipo 1. Para este ejemplo, imaginaremos que cada Tipo 1 tiene una potencia diferente que podemos ordenar.


In [37]:
poke_df = pd.read_csv('../../data/Pokemon.csv', encoding='latin1', index_col=0)
poke_df.head()

,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Stage,Legendary
#,,,,,,,,,,,,
1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,2,False
3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,3,False
4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False
5,Charmeleon,Fire,NaN,405,58,64,58,80,65,80,2,False


In [38]:
poke_df.columns

Index(['Name', 'Type 1', 'Type 2', 'Total', 'HP', 'Attack', 'Defense',
       'Sp. Atk', 'Sp. Def', 'Speed', 'Stage', 'Legendary'],
      dtype='object')

## EJERCICIO

Imprime por pantalla una lista ordenada (o Series o lo que sea) que identifique cada valor de "Type 1" con el número de veces que aparece.

In [39]:
poke_df['Type 1'].value_counts()

Water       28
Normal      22
Poison      14
Bug         12
Grass       12
Fire        12
Electric     9
Rock         9
Psychic      8
Ground       8
Fighting     7
Dragon       3
Ghost        3
Fairy        2
Ice          2
Name: Type 1, dtype: int64

## Estableciendo un orden personalizado

En general, no existe un módulo o función genérica para mapear y transformar estas variables en representaciones numéricas basadas en el orden automáticamente. Por lo tanto, podemos usar un mapa de codificación personalizado basado en un diccionario, tal como se muestra a continuación:

In [40]:
type_1_map = pd.Series({'Bug': 1, 'Water': 2, 'Rock': 3, 'Normal': 4, 'Fighting': 5, 'Grass': 6, 'Poison': 7,
       'Fire': 8, 'Ghost': 9, 'Fairy': 10, 'Electric': 11, 'Dragon':12, 'Ground':13,
       'Psychic':14, 'Ice':15})

# map the values to the dataframe

poke_df['type_1_num'] = poke_df['Type 1'].map(type_1_map)
poke_df.head()

,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Stage,Legendary,type_1_num
#,,,,,,,,,,,,,
1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False,6
2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,2,False,6
3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,3,False,6
4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False,8
5,Charmeleon,Fire,NaN,405,58,64,58,80,65,80,2,False,8


De este modo, por ejemplo, somos capaces de asignar la relación que nosotros queramos en función de su valor.

## EJERCICIO

Créate una nueva columna "type_2_num" que traduzca los tipos de "Type 2" a números entendibles por los futuros algoritmos. En este caso, a diferencia del caso anterior, el mapeo tendrá que hacer referencia a algo tangible que realmente exprese algo que asigne un valor coherente al tipo. Este mapeo deberá asignar los valores 1, 2, 3... a cada tipo en función de su ordenación por las características de ataque. Es decir, aquel tipo cuya suma de "Attack", "Sp. Atk" y "Speed" sea mayor, se mapeará como 1, el siguiente se mapeará como 2, y asi hasta el último tipo. Piensa además cómo mapearás aquellos tipos no informados, ya que no todos los Pokémon tienen "Type 2".

In [41]:
poke_df['suma'] = poke_df['Attack'] + poke_df['Sp. Atk'] + poke_df['Speed']
poke_df = poke_df.fillna('')

poke_type_2 = poke_df.groupby('Type 2').agg({'suma': 'max'})
poke_type_2
poke_type_2 = poke_type_2.sort_values(by='suma', ascending=False)
poke_type_2
poke_type_2 = poke_type_2.reset_index(drop=False).drop('suma', axis=1)
poke_type_2 = poke_type_2.reset_index(drop=False).set_index('Type 2')
poke_type_2['index'] + 1

# poke_df['type_2_num'] = poke_df['Type 2'].map(poke_type_2['index'] + 1).fillna(0).astype(int)
poke_df['type_2_num'] = poke_df['Type 2'].map(poke_type_2['index'] + 1)
poke_df

,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Stage,Legendary,type_1_num,suma,type_2_num
#,,,,,,,,,,,,,,,
1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False,6,159,3
2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,2,False,6,202,3
3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,3,False,6,262,3
4,Charmander,Fire,,309,39,52,43,60,50,65,1,False,8,177,1
5,Charmeleon,Fire,,405,58,64,58,80,65,80,2,False,8,224,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147,Dratini,Dragon,,300,41,64,45,50,50,50,1,False,12,164,1
148,Dragonair,Dragon,,420,61,84,65,70,70,70,2,False,12,224,1
149,Dragonite,Dragon,Flying,600,91,134,95,100,100,80,3,False,12,314,2


# Transformado variables Categóricas

En el ejemplo anterior, estábamos poniéndonos en el supuesto de que las variables de tipo tenían un significado tal que podíamos establecer un orden de relación (aunque en la realidad no fuese así). Por ejemplo, también podríamos haber utilizado otros ejemplos como tallajes de ropa. Ahora realizaremos los tratamientos pertinentes para aquellas variables categóricas puras, es decir, que no tienen una relación entre sí.

## One-Hot Encoder

## EJERCICIO

#### 1.

Antes hemos supuesto que "Type 1" y "Type 2" seguían cierta lógica. Sin embargo, esto no es así. Al hacer un mapeo, ya sea a mano a un número o mediante LabelEncoder, se está asignando una relación de forma indirecta. Por ello, en el notebook anterior habíamos visto cómo podíamos luchar contra esto. Realiza las operaciones necesarias para transformar nuestro dataset de modo que se muestre la información de "Type 1", "Type 2" y "Legendary" adaptada para ser usada con un algoritmo pero sin establecer ningún orden de preferencia entre los valores de los mismos.

Termina juntando tanto las nuevas columnas como el resto de datos del dataset original en un nuevo dataset.

Adicionalmente, almacena en una nueva variable llamada ``df_ej_1`` el DataFrame que has obtenido pero eliminando las columnas de estadísticas: "Total", "HP", "Attack", "Defense", "Sp. Atk", "Sp. Def", "Speed" y "Stage", para que podamos utilizarlo en el siguiente ejercicio.

In [101]:
poke_df = pd.read_csv('../../data/Pokemon.csv', encoding='latin1', index_col=0)

In [102]:
poke_df['Legendary'] = poke_df['Legendary'].astype(str)

one_hot_df = pd.get_dummies(poke_df[['Type 1', 'Type 2', 'Legendary']])
df_ej_1 = poke_df.join(one_hot_df).copy()
df_ej_1

,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,...,Type 2_Grass,Type 2_Ground,Type 2_Ice,Type 2_Poison,Type 2_Psychic,Type 2_Rock,Type 2_Steel,Type 2_Water,Legendary_False,Legendary_True
#,,,,,,,,,,,,,,,,,,,,,
1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,...,0,0,0,1,0,0,0,0,1,0
2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,...,0,0,0,1,0,0,0,0,1,0
3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,...,0,0,0,1,0,0,0,0,1,0
4,Charmander,Fire,NaN,309,39,52,43,60,50,65,...,0,0,0,0,0,0,0,0,1,0
5,Charmeleon,Fire,NaN,405,58,64,58,80,65,80,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147,Dratini,Dragon,NaN,300,41,64,45,50,50,50,...,0,0,0,0,0,0,0,0,1,0
148,Dragonair,Dragon,NaN,420,61,84,65,70,70,70,...,0,0,0,0,0,0,0,0,1,0
149,Dragonite,Dragon,Flying,600,91,134,95,100,100,80,...,0,0,0,0,0,0,0,0,1,0


In [60]:
df_ej_1 = df_ej_1.drop(["Total", "HP", "Attack", "Defense", "Sp. Atk", "Sp. Def", "Speed", "Stage"], axis=1)
df_ej_1

,Name,Type 1,Type 2,Legendary,Type 1_Bug,Type 1_Dragon,Type 1_Electric,Type 1_Fairy,Type 1_Fighting,Type 1_Fire,...,Type 2_Grass,Type 2_Ground,Type 2_Ice,Type 2_Poison,Type 2_Psychic,Type 2_Rock,Type 2_Steel,Type 2_Water,Legendary_False,Legendary_True
#,,,,,,,,,,,,,,,,,,,,,
1,Bulbasaur,Grass,Poison,False,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
2,Ivysaur,Grass,Poison,False,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
3,Venusaur,Grass,Poison,False,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
4,Charmander,Fire,NaN,False,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
5,Charmeleon,Fire,NaN,False,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147,Dratini,Dragon,NaN,False,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
148,Dragonair,Dragon,NaN,False,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
149,Dragonite,Dragon,Flying,False,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


#### 2.

Ahora, si nos diesen 2 nuevas entradas y nos pidieran que las categorizáramos como hemos hecho las anteriores, ¿cómo lo harías?

Debes tener claro que, en los problemas de Machine Learning, los datos deben tener la misma estructura, ya sean para entrenar el modelo o para predecir, por lo que en el DataFrame que nos creemos con estas entradas deberá tener las mismas columnas que el acabamos de sacar.

Hazlo con las siguientes 2 entradas:

In [61]:
new_poke_df = pd.DataFrame([['PikaZoom', 'Bug', np.nan, True], 
                           ['CharMyToast', 'Water', 'Poison', False]],
                           columns=['Name', 'Type 1', 'Type 2', 'Legendary'])

new_poke_df['Legendary'] = new_poke_df['Legendary'].astype(str)

one_hot_df_2 = pd.get_dummies(new_poke_df[['Type 1', 'Type 2', 'Legendary']])
new_poke_df = new_poke_df.join(one_hot_df_2)
new_poke_df

for col in df_ej_1.columns:
    if col not in new_poke_df.columns:
        new_poke_df[col] = 0

new_poke_df[df_ej_1.columns]


# df_ej_1 = poke_df.join(one_hot_df).copy()

,Name,Type 1,Type 2,Legendary,Type 1_Bug,Type 1_Dragon,Type 1_Electric,Type 1_Fairy,Type 1_Fighting,Type 1_Fire,...,Type 2_Grass,Type 2_Ground,Type 2_Ice,Type 2_Poison,Type 2_Psychic,Type 2_Rock,Type 2_Steel,Type 2_Water,Legendary_False,Legendary_True
0,PikaZoom,Bug,NaN,True,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,CharMyToast,Water,Poison,False,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0


#### 3.

Parece fácil, ¿no? Pues demos una vuelta más, ¿y si nos dieran las siguientes entradas? ¿Cómo lo harías?

_Tip: ¿no ves nada raro? Fíjate en los tipos de estos "Pokémon"_

In [46]:
new_poke_df_3 = pd.DataFrame([['PikaPika', 'Candy', np.nan, True],
                              ['Charlador', 'Water', 'Human', False]],
                             columns=['Name', 'Type 1', 'Type 2', 'Legendary'])

No todas las columnas están en el DataFrame de lo que hemos creado a partir del dataset original, por eso nos dará error en el momento que intentásemos atacar al algoritmo con algo distinto a lo que ha visto en el entrenamiento.

¿Qué podemos hacer? Bueno, podríamos filtrar los valores inicialmente y luego hacer la conversión; o podríamos filtrar las columnas después, quedando el vector de 0 para esta configuración. En este caso, haremos esta última.

In [73]:
new_poke_df_3_ohe = pd.get_dummies(new_poke_df_3[["Type 1", "Type 2", "Legendary"]])
new_poke_df_3_ohe

,Type 1_Candy,Type 1_Water,Type 2_Human,Legendary_False,Legendary_True
0,1,0,0,0,1
1,0,1,1,1,0


In [75]:
reduced_cols = [col for col in new_poke_df_3_ohe.columns if col in df_ej_1.columns]
new_poke_df_3_ohe = new_poke_df_3_ohe.loc[:, reduced_cols]
new_poke_df_3_ohe

,Type 1_Water,Legendary_False,Legendary_True
0,0,0,1
1,1,1,0


In [47]:
for col in df_ej_1.columns:
    if col not in new_poke_df_3_ohe.columns:
        new_poke_df_3_ohe[col] = 0
        
new_poke_df_3_ohe

,Legendary,Type 1_Water,Name,Type 1,Type 2,Type 1_Bug,Type 1_Dragon,Type 1_Electric,Type 1_Fairy,Type 1_Fighting,...,Type 2_Grass,Type 2_Ground,Type 2_Ice,Type 2_Poison,Type 2_Psychic,Type 2_Rock,Type 2_Steel,Type 2_Water,Legendary_False,Legendary_True
0,True,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,False,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Usando el objeto de OHE

Como piensan en todo, scikit-learn tiene un objeto que se encarga de esto por nosotros, donde le podremos explicar cómo tratar los valores que no haya visto en el entrenamiento (como en nuestro caso).

Ojo, tenemos que gestionar los nulos y el índice para hacer el join final. Aquí te dejo la [documentación](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html)

In [99]:
new_poke_df = poke_df[["Name", "Type 1", "Type 2", "Legendary"]].fillna("")
df_sk_ohe = new_poke_df[["Type 1", "Type 2", "Legendary"]]

dfa = pd.DataFrame(enc_fit.transform(df_sk_ohe).toarray())
dfa.columns = enc_fit.get_feature_names()
# dfa

In [123]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder


new_poke_df = poke_df[["Name", "Type 1", "Type 2", "Legendary"]].fillna("")
df_sk_ohe = new_poke_df[["Type 1", "Type 2", "Legendary"]]

# Creamos el encoder:
enc = OneHotEncoder(handle_unknown='ignore', dtype='int32')
# Entrenamos:
enc_fit = enc.fit(df_sk_ohe)

# Convertimos:
df_sk_ohe = pd.DataFrame(enc_fit.transform(df_sk_ohe).toarray())
# Ponemos los nombres de las columnas:
df_sk_ohe.columns = enc_fit.get_feature_names()
# Y lo juntamos con la parte original:
# df_sk = new_poke_df.reset_index(drop=True)
# df_ohe = new_poke_df.join(df_sk_ohe)
# df_ohe
df_sk_ohe

,x0_Bug,x0_Dragon,x0_Electric,x0_Fairy,x0_Fighting,x0_Fire,x0_Ghost,x0_Grass,x0_Ground,x0_Ice,...,x1_Grass,x1_Ground,x1_Ice,x1_Poison,x1_Psychic,x1_Rock,x1_Steel,x1_Water,x2_False,x2_True
0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,1,0
1,0,0,0,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,1,0
2,0,0,0,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,1,0
3,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
147,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
148,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
149,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [124]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder


new_poke_df = new_poke_df_3[["Name", "Type 1", "Type 2", "Legendary"]].fillna("")
df_sk_ohe = new_poke_df_3[["Type 1", "Type 2", "Legendary"]].fillna("")

# Convertimos:
df_sk_ohe = pd.DataFrame(enc_fit.transform(df_sk_ohe).toarray())
# Ponemos los nombres de las columnas:
df_sk_ohe.columns = enc_fit.get_feature_names()
# Y lo juntamos con la parte original:
new_poke_df = new_poke_df.reset_index(drop=True)
df_ohe = new_poke_df.join(df_sk_ohe)
df_ohe

,Name,Type 1,Type 2,Legendary,x0_Bug,x0_Dragon,x0_Electric,x0_Fairy,x0_Fighting,x0_Fire,...,x1_Grass,x1_Ground,x1_Ice,x1_Poison,x1_Psychic,x1_Rock,x1_Steel,x1_Water,x2_False,x2_True
0,PikaPika,Candy,,True,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,Charlador,Water,Human,False,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


Y ahora probamos con los dos ejemplos anteriores:

## Codificación Dummy

### EJERCICIO

Ahora, vamos a replicar lo que hemos hecho en el primer ejercicio de One-Hot Encoder, pero con la codificación Dummy. ¿Cómo lo hacíamos? ¿Podemos repetir lo mismo que hicimos en el primer caso?

¿Y con el objeto de scikit-learn?

Te recuerdo que aquí tienes la [documentación](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) para ver cómo hacerlo

No se puede directamente, necesitaríamos gestionar que solo pudiera tomar valores que ya existen, y añadir un nuevo valor para gestionar esos valores y mapearlos a esa columna, pero en ese caso no nos aportaría nada nuevo respecto a utilizar el OHE.

### EJERCICIO

Al usar el OHE, hemos visto que se crean muchas columnas nuevas. Prueba, sobre el dataset original de los Pokémon, a hacer el OHE y, después, aplicar un PCA. ¿Cuántas columnas tengo tras el OHE? ¿Con cuántas podría expresar más del 80% de la información de este dataset?


Varianza explicada por acumulación de las componentes:
[0.20362986 0.32263278 0.42147774 0.49651975 0.5567316  0.61168881
 0.66283419 0.70703076 0.74369364 0.77980737 0.81129181 0.84198622
 0.86714434 0.88987824 0.90974148 0.92637664 0.93933429 0.95015122
 0.95925194 0.96740944 0.97514913 0.9814434  0.98717555 0.99246492
 0.99671386 1.         1.         1.         1.        ]

Nº de componentes para > 80%:
11


,Name,Type 1,Type 2,Legendary,0,1,2,3,4,5,6,7,8,9,10
0,Bulbasaur,Grass,Poison,False,0.904843,-0.582403,-0.612236,-0.455880,0.002604,0.088384,-0.451289,0.018544,-0.007996,0.035911,0.128854
1,Ivysaur,Grass,Poison,False,0.904843,-0.582403,-0.612236,-0.455880,0.002604,0.088384,-0.451289,0.018544,-0.007996,0.035911,0.128854
2,Venusaur,Grass,Poison,False,0.904843,-0.582403,-0.612236,-0.455880,0.002604,0.088384,-0.451289,0.018544,-0.007996,0.035911,0.128854
3,Charmander,Fire,,False,-0.526067,0.096986,-0.275578,0.151130,0.431032,-0.315204,-0.250552,-0.605769,-0.111519,-0.138018,-0.183757
4,Charmeleon,Fire,,False,-0.526067,0.096986,-0.275578,0.151130,0.431032,-0.315204,-0.250552,-0.605769,-0.111519,-0.138018,-0.183757
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,Dratini,Dragon,,False,-0.449261,0.041750,-0.184089,0.032432,0.026332,-0.048938,0.021462,0.053378,0.036768,0.012536,0.126886
147,Dragonair,Dragon,,False,-0.449261,0.041750,-0.184089,0.032432,0.026332,-0.048938,0.021462,0.053378,0.036768,0.012536,0.126886
148,Dragonite,Dragon,Flying,False,0.655643,0.455691,0.307955,0.265674,0.351328,0.114527,-0.053377,0.079542,0.160716,-0.020152,0.291246
149,Mewtwo,Psychic,,True,-0.440439,0.180664,-0.186605,0.266275,0.539417,-0.230595,-0.219031,0.617251,0.205522,1.116612,-0.218780


## Función Hash

Volvamos ahora al dataset de ventas de juegos. 

In [162]:
df_videojuegos =  pd.read_csv('../../data/vgsales.csv')
df_videojuegos.head()

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


In [163]:
print('Nº Total de Géneros: ' + str(len(df_videojuegos.Genre.unique())))
print(df_videojuegos.Genre.sort_values().unique())

Nº Total de Géneros: 12
['Action' 'Adventure' 'Fighting' 'Misc' 'Platform' 'Puzzle' 'Racing'
 'Role-Playing' 'Shooter' 'Simulation' 'Sports' 'Strategy']


Podemos ver que hay un total de 12 géneros de videojuegos. Si usáramos un esquema de codificación One-Hot Encoding con la variable "Genre", terminaríamos teniendo 12 columnas binarias. En su lugar, ahora usaremos la codificación Hash, que ya hemos visto con anterioridad.

### EJERCICIO

Utiliza la función de _hasheo_ para codificar los valores del género (variable "Genre"). ¿Cuánto ocupa la variable original? ¿Y el objeto